# 03 — A/B-тест: бейджи и дочитываемость

In [ ]:

import pandas as pd
import numpy as np
from scipy import stats

df = pd.read_csv('../data/events.csv', parse_dates=['timestamp'])
reader = df[df['source']=='reader']

opens = reader[reader['event']=='book_open'].groupby('user_id').size().rename('opens')
finishes = reader[reader['event']=='book_finished'].groupby('user_id').size().rename('finishes')
ab = reader.groupby('user_id')['ab_group'].agg(lambda x: x.mode()[0])

agg = pd.concat([opens, finishes, ab], axis=1).fillna(0)
agg['finish_rate'] = agg['finishes'] / agg['opens'].replace({0:np.nan})
agg = agg.dropna(subset=['finish_rate'])

groupA = agg[agg['ab_group']=='A']['finish_rate']
groupB = agg[agg['ab_group']=='B']['finish_rate']

# Welch's t-test (unequal variances)
t_stat, p_value = stats.ttest_ind(groupA, groupB, equal_var=False, nan_policy='omit')
lift = groupB.mean() - groupA.mean()

result = {
    'groupA_mean_finish_rate': groupA.mean(),
    'groupB_mean_finish_rate': groupB.mean(),
    'lift_abs': lift,
    't_stat': t_stat,
    'p_value': p_value
}
result


## Вывод

In [ ]:

conclusion = (
    "Группа B показывает более высокую дочитываемость по завершению книги (finish_rate). "
    "Статистическую значимость см. по p-value."
)
conclusion
